In [1]:
%matplotlib inline

In [2]:
run SnowBoard_Jury_new.py

Index(['Mesa_Negligent', 'damages', 'Liability', 'gender', 'damages_word',
       'Scenario', 'Dunn_negligent', 'perc_calc', 'Start Date', 'End Date',
       'mm_perc'],
      dtype='object')
Mesa_Negligent     True
damages            True
Liability          True
gender             True
damages_word       True
Scenario           True
Dunn_negligent     True
perc_calc          True
Start Date        False
End Date          False
mm_perc            True
dtype: bool
True
False
Mesa_Negligent    False
damages           False
Liability         False
gender            False
damages_word      False
Scenario          False
Dunn_negligent     True
perc_calc         False
Start Date        False
End Date          False
mm_perc           False
dtype: bool
Mesa_Negligent    False
damages           False
Liability         False
gender            False
damages_word      False
Scenario          False
Dunn_negligent     True
perc_calc         False
Start Date        False
End Date          False
mm_pe

C:\Users\Pruthwiraj\Desktop\Jury_Q2\Jury_Project_Q2\SnowBoard_Jury_new.py:258: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  'Path'])


StartDate                                              0
EndDate                                                0
Was_snowboard_sold_McNeil_defective_14                 0
Is_substantial_factor_McNeil_injuries_14             618
Non_economic_damages_McNeil_suffered_14              637
Was_McNeil_negligent                                 489
McNeil_negligence_substantial_factor_for_injuries    489
Percentage_of_responsibility_X5                      679
Percentage_of_responsibility_McNeil                  679
Was_snowboard_sold_McNeil_defective_58                 0
Is_substantial_factor_McNeil_injuries_58             629
Economic_damages_McNeil_suffer_58                    656
Non_economic_damages_McNeil_suffered_58              656
Q40                                                   33
Path                                                   0
dtype: int64
StartDate                                            False
EndDate                                              False
Was_snowboard_